Thực hiện cào dữ liệu với các thông tin sau:

- Link : Đường dẫn của job.
- Location: Địa điểm.
- Update Date: Ngày up CV.
- Deadline: Hạn nộp CV.
- Salary: Lương.
- Experience: Kinh nghiệm.
- Rank: Cấp bậc.
- Job type: Loại ngành nghề.
- Briefcase:
- Welfare: Phúc lợi.
- Job Description: Mô tả công việc.
- Job Requirement: Yêu cầu công việc.
- Job Location: Địa điểm làm việc.
- Other Information: Thông tin khác.

_executable_path_ mọi người tải về chromedriver.exe phù hợp với phiên bản của Chrome đang xài rồi dán đường dẫn chứa chromedriver.exe để cào dữ liệu. Còn trình duyệt khác thì không rõ.

In [3]:
! pip install selenium

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.7 MB 4.8 MB/s eta 0:00:02
   ------ --------------------------------- 1.6/9.7 MB 4.4 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/9.7 MB 4.3 MB/s eta 0:00:02
   -------------- ------------------------- 3.4/9.7 MB 4.3 MB/s eta 0:00:02
   ------------------ --------------------- 4.5/9.7 MB 4.3 MB/s eta 0:00:02
   --------------------- ------------------ 5.2/9.7 MB 4.2 MB/s eta 0:00:02
   ----------------------- ---------------- 5.8/9.7 MB 4.1 MB/s eta 0:00:01
   ---------------------------- ----------- 6.8/9.7 MB 4.1 MB/s eta 0:00:01
   -------------------------------- ------- 7.9/9.7 MB 4.2 MB/s eta 0:00:01
   ------------------------------------ --- 8.9/9.7 MB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 4.3 MB/s eta 0:00:00


In [4]:
# Necessary Packages 
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [ ]:
BASE_URL = "https://careerviet.vn/viec-lam/"

# Set up the Chrome Options
executable_path = "D:/KHTN/datascienceintro/chromedriver-win64/chromedriver-win64/chromedriver.exe"

In [ ]:
def crawlJobLinks(pageStart: int,pageEnd:int) -> list:
    chrome_options=Options()
    prefs = {
    "profile.default_content_setting_values.geolocation": 2  # 2 means block
    }
    chrome_options.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(executable_path=executable_path,options=chrome_options)
    job_links_list=[]
    for page in range(pageStart,pageEnd+1):
        url= f"{BASE_URL}tat-ca-viec-lam-trang-{page}-vi.html"
        driver.get(url)
        time.sleep(random.randint(5, 10))
        for job in driver.find_elements(By.CSS_SELECTOR,'h2 > .job_link'):
            job_links_list.append(job.get_attribute('href'))
        
    driver.quit()
    return job_links_list

In [ ]:
def getInfoJob(job_url) -> list:
    chrome_options=Options()
    prefs = {
    "profile.default_content_setting_values.geolocation": 2  # 2 means block
    }
    chrome_options.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(executable_path,options=chrome_options)
    
    driver.get(job_url)
    time.sleep(random.randint(5, 10))
    #Địa điểm
    try:
        location_element=driver.find_element(By.CSS_SELECTOR,".map p a")
        location=location_element.text
    except:
        location="Not Found"

    #Ngày cập nhật
    try:
        update_date=driver.find_element(By.XPATH,"//strong[contains(text(),'Ngày cập nhật')]/following-sibling::p").text
    except:
        update_date="Not Found"

    #Loại ngành nghề
    try:
        li_briefcase_element = driver.find_element(By.CSS_SELECTOR,"li:has(.mdi-briefcase)")
        a_briefcase_elements=li_briefcase_element.find_elements(By.CSS_SELECTOR,"a")
        list_briefcase=""
        for link in a_briefcase_elements:
            list_briefcase+=link.text+", "   
        list_briefcase=list_briefcase.rstrip(", ")
    except:
        list_briefcase="Not Found"

    #Hình thức
    try:
        job_type=driver.find_element(By.XPATH,"//strong[contains(text(),'Hình thức')]/following-sibling::p").text
    except:
        job_type="Not Found"

    #Lương
    try:
        salary=driver.find_element(By.XPATH,"//strong[contains(text(),'Lương')]/following-sibling::p").text
    except:
        salary="Not Found"

    #Kinh nghiệm
    try:
        experience=driver.find_element(By.XPATH,"//strong[contains(text(),'Kinh nghiệm')]/following-sibling::p").text
    except:
        experience="Not Found"

    #Cấp bậc
    try:
        rank=driver.find_element(By.XPATH,"//strong[contains(text(),'Cấp bậc')]/following-sibling::p").text
    except:
        rank="Not Found"

    #Deadline
    try:
        deadline=driver.find_element(By.XPATH,"//strong[contains(text(),'Hết hạn nộp')]/following-sibling::p").text
    except:
        deadline="Not Found"

    #Phúc lợi
    try:
        li_elements=driver.find_elements(By.CSS_SELECTOR,".welfare-list > li")
        list_welfare=""
        for li in li_elements:
            list_welfare+=li.text+", "
        list_welfare=list_welfare.rstrip(", ")
    except:
        job_description="Not Found"

    #Mô tả công việc
    try:
        h2_element=driver.find_element(By.CSS_SELECTOR,".reset-bullet")
        job_description=h2_element.text
    except:
        job_description="Not Found"


    #Yêu cầu công việc
    try:
        h2_element=driver.find_element(By.CSS_SELECTOR,".reset-bullet + .reset-bullet")
        job_requirement=h2_element.text
    except:
        job_requirement="Not Found"

    #Địa điểm làm việc
    try:
        place_element=driver.find_element(By.CSS_SELECTOR,".place-name .place")
        job_location=place_element.text
    except:
        job_location="Not Found"

    #Thông tin khác
    try:
        other_info_element=driver.find_element(By.CSS_SELECTOR,".content_fck")
        other_info=other_info_element.text
    except:
        other_info="Not Found"

    driver.quit()


    job_information=[job_url,location,update_date,deadline,salary,experience,rank,job_type,list_briefcase,list_welfare,job_description,job_requirement,job_location,other_info]

    return job_information 

In [ ]:
def crawlJobInformation(job_urls) -> pd.DataFrame:
    job_information_list=[]
    for job_url in job_urls:
        job_information=getInfoJob(job_url)
        job_information_list.append(job_information)
    df=pd.DataFrame(job_information_list,columns=["Link","Location","Update Date","Deadline","Salary","Experience","Rank","Job Type","Briefcase","Welfare","Job Description","Job Requirement","Job Location","Other Information"])
    return df

In [13]:
pageStart=1
pageEnd=3
job_urls=crawlJobLinks(pageStart=1,pageEnd=3)

C:\Users\queol\AppData\Local\Temp\ipykernel_28620\3593696083.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=executable_path,options=chrome_options)


In [ ]:

df=crawlJobInformation(job_urls)


In [ ]:
df.to_csv("job_information.csv",index=False,encoding="utf-8-sig")